In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.models import Model
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
import pdb
from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, Embedding, merge, Dropout, Dense, Input, Flatten
from google.colab import files
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import math

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

categories = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc']
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True, categories=categories)
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
texts = []
labels = newsgroups_train.target
texts = newsgroups_train.data
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data
y_train = labels
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


In [ ]:
GLOVE_DIR = "C:\\Users\\pythonrunfiles"
word_index = tokenizer.word_index
embeddings_index = {}
f = open('/content/drive/My Drive/Colab Notebooks/Data/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

model = Sequential()
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1 = Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(19, activation='softmax')(l_dense)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 1000, 100)         13187400  
_________________________________________________________________
conv1d (Conv1D)              (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 128)           82048 

In [ ]:
model.fit(x_train, y_train, validation_split= 0.2, epochs=20, batch_size=128)


Epoch 1/20
55/55 [==============================] - 13s 93ms/step - loss: 2.9808 - acc: 0.0739 - val_loss: 2.5750 - val_acc: 0.1429
Epoch 2/20
55/55 [==============================] - 5s 86ms/step - loss: 2.3167 - acc: 0.2277 - val_loss: 1.8056 - val_acc: 0.3720
Epoch 3/20
55/55 [==============================] - 5s 85ms/step - loss: 1.6273 - acc: 0.4299 - val_loss: 1.5352 - val_acc: 0.4554
Epoch 4/20
55/55 [==============================] - 5s 85ms/step - loss: 1.2055 - acc: 0.5774 - val_loss: 1.2548 - val_acc: 0.5971
Epoch 5/20
55/55 [==============================] - 5s 85ms/step - loss: 0.9243 - acc: 0.6715 - val_loss: 1.3368 - val_acc: 0.5629
Epoch 6/20
55/55 [==============================] - 5s 85ms/step - loss: 0.6676 - acc: 0.7836 - val_loss: 0.7888 - val_acc: 0.7377
Epoch 7/20
55/55 [==============================] - 5s 85ms/step - loss: 0.4934 - acc: 0.8365 - val_loss: 0.7220 - val_acc: 0.7737
Epoch 8/20
55/55 [==============================] - 5s 85ms/step - loss: 0.3547 - 

In [ ]:
#@title ## Auto Annotations!
#@markdown Try-it-out by pasting or typing your text below:
#X = df.blind_assesment.values
Type_here = "There are several major rules, which differ only slightly. The \"Official Baseball Rules\" govern all professional play in the United States and Canada, including the World Baseball Classic. The complete rules are published as the Official Baseball Rules at MLB.com, the official web site of Major League Baseball in the United States and Canada. The rules are also published in book form in North America by the Sporting News. Many amateur and youth leagues use the \"OBR\" with only a few modifications for safety.  Other slightly modified versions of the \"Official Baseball Rules\" are used in competitions operated by the International Baseball Federation (IBAF), including the Olympics and the World Cup, most leagues outside North America (such as Japan's Nippon Professional Baseball), and youth baseball organizations such as Little League, PONY League, and Cal Ripken League.  The baseball rulebook of the National Collegiate Athletic Association (NCAA), aside from governing the games of that organization's members, is also used by several other competitions involving college-aged players.  U.S. high school and high school-age baseball is governed by the rules of the National Federation of State High School Associations (NFHS)."#@param {type:"string"}
maxlen = 1000
tokenizer = Tokenizer(num_words= 400000)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences([Type_here])
data = pad_sequences(sequences,padding='post', maxlen=maxlen)
indices = np.arange(data.shape[0])
data = data[indices]
Xtst = data

y_pred = model.predict(Xtst)

class_idx = np.argmax(y_pred[0])
class_output = model.output[:, class_idx]
last_conv_layer = model.get_layer("conv1d")

grads = K.gradients(class_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads)
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([Xtst])

    
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)#normalise values in the prediction
norm_len = maxlen/last_conv_layer.output_shape[1] # find the ratio of the text vs the conv layer length

html = ""

html += "<span><h3>Based on the description, the model believes that this is {}.".format(categories[int((np.argmax(y_pred)))])
html += "<small></small></h3></span>".format(abs(((y_pred[0][0]*100)-50)*2))
for j,i in enumerate(tokenizer.sequences_to_texts(Xtst)[0].split()):
  html += "<span style='background-color:rgba({},0,50,{})'>{} </span>".format(heatmap[math.floor(j/norm_len)]*255,heatmap[math.floor(j/norm_len)]-0.3,i)

HTML(html)